In [1]:
import requests
from datetime import date
from dateutil.rrule import rrule, DAILY
import pandas as pd

# scrape airport code

In [40]:
code=pd.read_html('https://www.leonardsguide.com/us-airport-codes.shtml')

In [41]:
airport_code=code[0]

In [42]:
code2=pd.DataFrame(airport_code)

In [45]:
code2.to_csv("airport_code.csv")

# flight data of all_state

In [47]:
aircode=pd.read_csv('airport_code3.csv')

In [51]:
iata=aircode['code']

In [52]:
a = date(2020, 12, 1)
b = date(2020, 12, 31)
day=[]
for dt in rrule(DAILY, dtstart=a, until=b):
    day.append(dt.strftime("%Y-%m-%d"))

In [53]:
len(iata)

190

In [54]:
len(day)

31

In [55]:
total_flight=[]
city=[]

for m in iata:
    city=m
    for i in day:
        params = {
            'access_key':'xxx',
            'flight_date':'{}'.format(i),
            #'dep_iata': 'DFW',
            'arr_iata': '{}'.format(m)
        }

        api_result = requests.get('http://api.aviationstack.com/v1/flights', params)
        api_response = api_result.json()
        flight=api_response['pagination']['total']
        
        df= pd.DataFrame({'city_iata':[city],'date':[i], 'total_flight':[flight]})
        total_flight.append(df)
        
df2 = pd.concat(total_flight)

In [56]:
df2.to_csv("city_flight_date_allstate.csv", sep=',',index=False)

In [57]:
df2=pd.read_csv('city_flight_date_allstate.csv')

In [58]:
df2['date']=pd.to_datetime(df2['date'], format='%Y-%m-%d')
df2['year'] = df2['date'].apply(lambda x: x.year)
df2['month'] = df2['date'].apply(lambda x: x.month)
df2['date'] = df2['date'].apply(lambda x: x.day)
df2=df2.groupby(['city_iata','year','month']).sum().reset_index()
df3=df2.drop(['date'], axis=1)

In [59]:
df3.to_csv("city_flight_month_allstate.csv")

# merge

In [60]:
flights=pd.read_csv("city_flight_month_allstate.csv")

In [61]:
flights.head()

,Unnamed: 0,city_iata,year,month,total_flight
0,0,ABE,2020,12,987
1,1,ABQ,2020,12,2883
2,2,ACK,2020,12,1250
3,3,ACY,2020,12,326
4,4,AGS,2020,12,1043


In [62]:
aircode.head()

,state,airport,code
0,AL,Birmingham International Airport,BHM
1,AL,Dothan Regional Airport,DHN
2,AL,Huntsville International Airport,HSV
3,AL,Mobile,MOB
4,AL,Montgomery,MGM


In [64]:
flights.shape

(189, 5)

In [65]:
aircode.shape

(190, 3)

In [66]:
cc=aircode.merge(flights,left_on='code',right_on='city_iata')

In [71]:
dd=cc.drop(['Unnamed: 0','city_iata'],axis=1)

In [72]:
dd.head()

,state,airport,code,year,month,total_flight
0,AL,Birmingham International Airport,BHM,2020,12,2874
1,AL,Dothan Regional Airport,DHN,2020,12,527
2,AL,Huntsville International Airport,HSV,2020,12,1636
3,AL,Mobile,MOB,2020,12,1076
4,AL,Montgomery,MGM,2020,12,782


In [73]:
dd.to_csv("flight_allstate.csv")